In [14]:
# ! curl https://s3.amazonaws.com/hr-testcases/597/assets/trainingdata.txt -o data/trainingdata.txt

In [133]:
# Enter your code here. Read input from STDIN. Print output to STDOUT
# create a df/itf per word
# doc freq inverse token freq
# create a tf idf vector per document
# check which vector is closest
from math import log
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV,RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial import distance
from collections import Counter
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text
from sklearn.utils import class_weight
import random
C = 9
with open('data/trainingdata.txt') as f:
    lines = f.read().split('\n')

def normalize(text):
    return text.lower()

y = []
data = []
for line in lines[1:-1]:
    c,line = line.split(' ',1)
    y.append(int(c))
    data.append(normalize(line))

# print(len(cnt))
# print(len([w for w in cnt if cnt[w]<=3]))
# print(len(text.ENGLISH_STOP_WORDS))
    
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=8000, stop_words='english')
vectorizer = vectorizer.fit(data)
X = vectorizer.transform(data)
y = np.array(y)
# X,y = uniquify(X,y)

# clf = LogisticRegression(solver='liblinear', multi_class='auto').fit(X, y)

cnt = Counter(y)
# print(cnt)

def k_nearest(X_,k):
    dist = cosine_similarity(X, X_).flatten()
    first = np.argsort(dist)[-k:]
    cs = [y[i] for i in first]
    cnt_ = Counter(cs)
    # cnt_ = Counter({c:n/cnt[c] for c,n in cnt_.items()})
    # print(cnt_)
    return cnt_.most_common(1)[0][0]


In [25]:
N = X.shape[0]

In [47]:
test_i = random.sample(range(N),N//10)

In [48]:
train_i = [i for i in range(N) if i not in test_i]

In [49]:
test = X[test_i, :]
train = X[train_i, :]

In [50]:
test_y = y[test_i]
train_y = y[train_i]

In [74]:
labels = np.unique(train_y)

In [81]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 labels,
                                                 train_y)

In [83]:
Counter(train_y)

Counter({1: 2578, 2: 1422, 4: 101, 6: 224, 5: 34, 7: 169, 8: 185, 3: 224})

In [84]:
class_weights

array([ 0.2393813 ,  0.43398383,  2.75502232,  6.11014851, 18.15073529,
        2.75502232,  3.65162722,  3.33581081])

In [85]:
class_weights={i:w for i,w in  zip(labels,class_weights)}

In [138]:
clf = LogisticRegression(class_weight=class_weights).fit(train, train_y)

In [134]:
# clf = RidgeClassifier(class_weight=class_weights).fit(train, train_y)

In [139]:
pred = clf.predict(test)
len((pred - test_y).nonzero()[0])/len(test_y)

0.05291970802919708

In [155]:
mis = (pred - test_y).nonzero()[0]

In [234]:
j = 13

In [235]:
i = test_i[mis[j]]

In [236]:
(vectorizer.transform([normalize(lines[i+1])]) == test[mis[j]]).toarray().all()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


True

In [237]:
test_y[mis[j]]

1

In [238]:
pred[mis[j]]

2

In [239]:
lines[i+1]

'1 transworld liquidating twt to distribute transworld corp liquidating trust said it expects to make an initial distribution to beneficiaries valued at dlrs per unit from the proceeds of the sale of hilton international co the value of the distribution assumes yesterday s closing price of ual s common stock of dlrs per share earlier ual announced that it completed the purchase of hilton international co for mln dlrs in cash and mln shares of ual inc common stock total value of the sale is about mln dlrs transworld said pursuant to the sale ual exercised its option to substitute cash for mln dlrs of debentures and shares of common stock transworld liquidating said each unit of beneficial interest in the trust will be allocated shares of ual common stock the aggregate value of the distribution is mln dlrs the balance of the cash in the trust will be held by the trust until april and will be used to satisfy all ouststanding liabilities and obligations of the trust after satisfaction of i

In [211]:
conf = np.zeros((C,C))

In [215]:
for j in mis:
    print(pred[j],test_y[j])
    conf[pred[j],test_y[j]]+=1

1 2
6 1
2 1
7 8
7 8
8 7
6 2
6 3
2 7
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 6
2 1
3 7
2 1
8 7
7 8


In [214]:
conf

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., 17.,  0.,  0.,  0.,  0.,  1.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.]])

In [137]:
# for k in range(20,40):
#     clf = KNeighborsClassifier(n_neighbors=k).fit(train, train_y)
#     pred = clf.predict(test)
#     print(k,len((pred - test_y).nonzero()[0])/len(test_y))

In [132]:
k=30
lines = """This is a document 
this is another document 
documents are seperated by newlines""".split('\n')
for line in lines:
    X_ = vectorizer.transform([normalize(line)])
    clf.predict(X_)
    print(k_nearest(X_,k))
    print(clf.predict(X_)[0])



1
2
1
2
2
2
